In [1]:
# 파일이름 : model_learning.ipynb
# 코드설명 : AutoML 기반 고무 배합의 물성값 및 불량여부 예측 AI 모델 학습 및 결과 저장
# 입/출력 : 물성별(물성값, 불량여부) 데이터 세트 / 예측 대상별 학습된 모델들
# 유의 사항 : 
# 1. 기본물성과 가류/점도를 한 번에 학습
# 2. 물성값 회귀모델 학습: regression, 불량여부 분류모델 학습 : binary로 설정
# 3. autogluon(0.8.2)과 python(3.9.18) 버전을 맞추어야 함
# 최종수정 : 2023년 11월 27일
# 제 작 자 : 홍민성 (mshong@micube.co.kr), 맹영준 (myj6223@micube.co.kr)
# Copyright : MICUBE Solution, Inc.

# python 버전 : 3.9.18
# autogluon 버전 : 0.8.2

In [2]:
# 관련 라이브러리 로드
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

import os
import random
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, recall_score, accuracy_score

# autoML
from autogluon.tabular import TabularPredictor
import time as time
# 동일 결과 재생성을 위한 랜덤성 고정
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [3]:
# SMAPE 계산 함수 정의
def smape_cal(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# 0을 제외한 MAPE 계산 함수 정의
def mape_non_zero(y_true, y_pred):
    mask = y_true != 0
    return np.mean(np.abs((y_pred[mask] - y_true[mask]) / y_true[mask]))

## 1. AutoML (Autogluon) 학습 및 모델 저장

### 1) 물성 값 예측을 위한 회귀(regression) 모델 학습 및 저장

In [4]:
%%time

# 물성값: regression, 불량여부: binary
problem_type_ = 'regression' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

# 3분절 교차검증
nfold = 3

if problem_type_ == 'regression':
    # 물성값 예측
    yCols=['HS','SG','TS','EB','MNY_MIN','MNY_T5','MNY_T35','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90']
    # yCols=['REHO_MIN','REHO_TS2']
    data_type_ = 'SFT_regr'
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT','MNY_RESULT','REHO_RESULT','SCR_RESULT']
    # yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'

for yCol in yCols:
    start = time.time()
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train['REAL_VAL'], test_size=0.2, random_state=seed)
    train_data = train.iloc[X_train.index].drop(['ID','TYPE'], axis=1)

    # # 모델 학습
    # predictor = TabularPredictor(
    #     label='REAL_VAL', 
    #     problem_type=problem_type_,  # 회귀 문제로 명시적 설정  binary, multiclass, regression, quantile
    #     eval_metric=None,           # 'accuracy' for binary/multiclass, 'root_mean_squared_error' for regression, and 'pinball_loss' for quantile.
    #     path=f'{data_type_}/{file_suffix}_{yCol}_models',  # 모델을 저장할 디렉토리의 전체 경로
    #     if problem_type_ == 'binary':
    #         sample_weight='balance_weight',
    # ).fit(
    #     train_data,
    #     presets='best_quality',     # most accurate overall predictor: 'best_quality', good quality with minimal disk usage : 'high_quality','optimize_for_deployment'
    #                                 # others : 'medium_quality', 'interpretable', 'ignore_text'
    #     time_limit=300,
    #     verbosity=0,
    #     num_bag_folds=nfold,            # n분절-교차검증
    #     # auto_stack=True,          # presets에 따라 자동 설정?
    #     # num_gpus='auto',          # default : auto
    # )

    # 모델 인자 설정
    model_args = {
        'label': 'REAL_VAL',
        'problem_type': problem_type_,
        'eval_metric': None,
        'path': f'{data_type_}/{file_suffix}_{yCol}_models',
    }
    
    # # 특정 모델만 사용 설정
    # hyperparameters = {
    #     'GBM': {},
    #     'XT': {},  # ExtraTrees 모델 설정
    #     'RF': {},  # RandomForest 모델 설정
    #     'XGB': {}  # XGBoost 모델 설정
    # }
    
    # 조건에 따른 추가 인자 설정
    if problem_type_ == 'binary':
        model_args['sample_weight'] = 'balance_weight'
    
    # 모델 학습
    predictor = TabularPredictor(**model_args).fit(
        train_data,
        presets='best_quality',
        time_limit=200, #300
        verbosity=0,
        num_bag_folds=nfold,
        num_gpus='auto',
        num_stack_levels=1,  # 스태킹 레벨을 1로 설정하여 L1까지만 학습
        dynamic_stacking=False,
        #hyperparameters=hyperparameters,
    )

    # 테스트 세트에 대한 예측
    y_pred = predictor.predict(X_test)

    res_df = pd.DataFrame()
    res_df[yCol+'_real'] = y_test
    res_df[yCol+'_pred'] = y_pred

    # 성능 평가
    if problem_type_ == 'regression':
        mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        zero_mape2 = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape2:.4f}, smape: {smape:.4f}")
    else:
        recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")

    # 학습된 모델의 성능 요약
    display(predictor.leaderboard())
    print(f'Excution time for {yCol} : {time.time() - start}s')
    print(f'='*80,'\n')    

HS!!!!
HS >> MAPE: 0.0204, zero_mape: 0.0204, smape: 0.0202


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-2.279489,root_mean_squared_error,20.926387,158.957199,0.001005,0.030382,3,True,10
1,RandomForestMSE_BAG_L2,-2.313382,root_mean_squared_error,20.433304,127.639054,0.952536,21.959915,2,True,8
2,LightGBM_BAG_L2,-2.341334,root_mean_squared_error,19.623910,109.549438,0.143143,3.870299,2,True,7
3,CatBoost_BAG_L2,-2.366563,root_mean_squared_error,19.646750,129.094756,0.165983,23.415616,2,True,9
4,LightGBMXT_BAG_L2,-2.386529,root_mean_squared_error,19.663720,109.680987,0.182952,4.001848,2,True,6
5,WeightedEnsemble_L2,-2.404370,root_mean_squared_error,16.801508,105.502207,0.000000,0.022247,2,True,5
6,LightGBMXT_BAG_L1,-2.471288,root_mean_squared_error,13.198840,90.275082,13.198840,90.275082,1,True,3
7,LightGBM_BAG_L1,-2.485216,root_mean_squared_error,1.040023,15.060994,1.040023,15.060994,1,True,4
8,KNeighborsDist_BAG_L1,-4.032636,root_mean_squared_error,2.562646,0.143883,2.562646,0.143883,1,True,2
9,KNeighborsUnif_BAG_L1,-4.429603,root_mean_squared_error,2.679259,0.199180,2.679259,0.199180,1,True,1


Excution time for HS : 205.89823365211487s

SG!!!!
SG >> MAPE: 0.0060, zero_mape: 0.0060, smape: 0.0060


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.014515,root_mean_squared_error,24.425097,145.867272,0.002000,0.055708,3,True,10
1,LightGBM_BAG_L2,-0.014564,root_mean_squared_error,23.630896,105.969613,0.220840,4.399409,2,True,8
2,CatBoost_BAG_L2,-0.014781,root_mean_squared_error,23.657391,132.330566,0.247335,30.760362,2,True,9
3,LightGBMXT_BAG_L2,-0.014801,root_mean_squared_error,23.954922,110.651793,0.544866,9.081589,2,True,7
4,WeightedEnsemble_L2,-0.014943,root_mean_squared_error,14.998777,94.890521,0.001000,0.042429,2,True,6
5,LightGBMXT_BAG_L1,-0.015132,root_mean_squared_error,3.980555,48.982494,3.980555,48.982494,1,True,3
6,LightGBM_BAG_L1,-0.015179,root_mean_squared_error,3.065727,45.574574,3.065727,45.574574,1,True,4
7,KNeighborsDist_BAG_L1,-0.032003,root_mean_squared_error,7.951496,0.291023,7.951496,0.291023,1,True,2
8,KNeighborsUnif_BAG_L1,-0.034000,root_mean_squared_error,8.161548,0.276830,8.161548,0.276830,1,True,1
9,CatBoost_BAG_L1,-0.053197,root_mean_squared_error,0.250731,6.445283,0.250731,6.445283,1,True,5


Excution time for SG : 209.13141632080078s

TS!!!!


...

Excution time for REHO_MAX : 235.56385350227356s

REHO_TS2!!!!
REHO_TS2 >> MAPE: 0.0525, zero_mape: 0.0525, smape: 0.0517


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-4.815563,root_mean_squared_error,83.330220,107.596982,0.003013,0.180204,3,True,5
1,LightGBMXT_BAG_L2,-4.816755,root_mean_squared_error,83.327206,107.416777,3.132507,59.030411,2,True,4
2,WeightedEnsemble_L2,-4.998716,root_mean_squared_error,80.197714,48.523976,0.003015,0.137610,2,True,3
3,LightGBMXT_BAG_L1,-5.115299,root_mean_squared_error,2.389072,47.561206,2.389072,47.561206,1,True,2
4,KNeighborsUnif_BAG_L1,-5.672625,root_mean_squared_error,77.805627,0.825161,77.805627,0.825161,1,True,1


Excution time for REHO_TS2 : 229.56408596038818s

REHO_TC90!!!!
REHO_TC90 >> MAPE: 0.0518, zero_mape: 0.0518, smape: 0.0512


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-11.124349,root_mean_squared_error,95.448385,97.048529,0.003066,0.171310,3,True,5
1,LightGBMXT_BAG_L2,-11.125567,root_mean_squared_error,95.445319,96.877219,2.629518,55.477272,2,True,4
2,WeightedEnsemble_L2,-11.576809,root_mean_squared_error,92.819812,41.554165,0.004011,0.154217,2,True,3
3,LightGBMXT_BAG_L1,-11.901785,root_mean_squared_error,1.687362,40.599977,1.687362,40.599977,1,True,2
4,KNeighborsUnif_BAG_L1,-13.119707,root_mean_squared_error,91.128439,0.799970,91.128439,0.799970,1,True,1


Excution time for REHO_TC90 : 234.6339294910431s

CPU times: total: 1h 48min 24s
Wall time: 40min 2s
